# Back Testing

- Import Statements

In [41]:
import numpy as np
from pandas import Timestamp
import datetime as dt
import pandas as pd
import yfinance as yf
import math
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt

In [96]:

def sanitize(df):    
    if df.empty:
        return
    if len(df.columns) > 0:
        common_names = {
            "Date": "date",
            "Time": "time",
            "Timestamp": "timestamp",
            "Datetime": "datetime",
            "Open": "open",
            "High": "high",
            "Low": "low",
            "Close": "close",
            "Adj Close": "adj_close",
            "Volume": "volume",
            "Dividends": "dividends",
            "Stock Splits": "split",
            "open_price": "open",
            "high_price": "high",
            "low_price": "low",
            "close_price": "close",
            "traded_quantity": "volume",
        }
        # Preemptively drop the rows that are all NaNs
        # Might need to be moved to AnalysisIndicators.__call__() to be
        #   toggleable via kwargs.
        # df.dropna(axis=0, inplace=True)
        # Preemptively rename columns to lowercase
        df.rename(columns=common_names, errors="ignore", inplace=True)
        
        col_types = {
            "open": float,
            "high": float,
            "low": float,
            "close": float,
        }
        
        df = df.astype(col_types)

        # Preemptively lowercase the index
        index_name = df.index.name
        if index_name is not None:
            df.index.rename(index_name.lower(), inplace=True)
        else:
            df.set_index(pd.DatetimeIndex(df['date']))
            
        return df
    else:
        raise AttributeError(f"[X] No columns!")

def _create_level_object(row, type):
    levels = []
    level = {}
    level["type"] = f"{type} open"
    level["level"] = np.round(row["open"], 2)
    levels.append(level)    
    
    level = {}
    level["type"] = f"{type} high"
    level["level"] = np.round(row["high"])
    levels.append(level)   
    
    level = {}
    level["type"] = f"{type} low"
    level["level"] = np.round(row["low"])   
    levels.append(level)   
    
    level = {}
    level["type"] = f"{type} close"
    level["level"] = np.round(row["close"])  
    levels.append(level)
    return levels

def _current_previous_levels(df, type):    
    levels = []
    
    level = _create_level_object(df.iloc[-1], f"current {type}")
    levels.extend(level)
    
    level = _create_level_object(df.iloc[-2], f"previous {type}")
    levels.extend(level)
    
    return levels

def monthly_levels(df):
    df_values = df.resample('M').agg({'open':'first', 'high':'max', 'low': 'min', 'close':'last'})    
    return _current_previous_levels(df_values, "month")

def weekly_levels(df):
    df_values = df.resample('W').agg({'open':'first', 'high':'max', 'low': 'min', 'close':'last'})    
    return _current_previous_levels(df_values, "week")

def firty_two_week_levels(df):
    levels = []    
    df['52W H'] = df['high'].rolling(window=252, center=False).max()
    df['52W L'] = df['low'].rolling(window=252, center=False).min()
    
    level = {}
    level["type"] = f"52 week high"
    level["level"] = np.round(df['52W H'].iloc[-1], 2)
    levels.append(level) 
    
    level = {}
    level["type"] = f"52 week low"
    level["level"] = np.round(df['52W L'].iloc[-1], 2)
    levels.append(level)
    
    return levels

def _support(df, index, n1, n2): 
    #n1 n2 before and after candle index    
    for i in range(index-n1+1, index+1):
        if(df['low'][i] > df['low'][i-1]):
            return False
        
    for i in range(index+1, index+n2+1):
        if(df['low'][i] < df['low'][i-1]):
            return False
    return True

def _resistance(df, index, n1, n2):
    #n1 n2 before and after candle index     
    for i in range(index-n1+1, index+1):
        if(df['high'][i] < df['high'][i-1]):
            return False
        
    for i in range(index+1, index+n2+1):
        if(df['high'][i] > df['high'][i-1]):
            return False
    return True

#method 1: fractal candlestick pattern
# determine bullish fractal 
def _is_support(df,i):  
  cond1 = df['low'][i] < df['low'][i-1]   
  cond2 = df['low'][i] < df['low'][i+1]   
  cond3 = df['low'][i+1] < df['low'][i+2]   
  cond4 = df['low'][i-1] < df['low'][i-2]  
  return (cond1 and cond2 and cond3 and cond4) 

# determine bearish fractal
def _is_resistance(df,i):  
  cond1 = df['high'][i] > df['high'][i-1]   
  cond2 = df['high'][i] > df['high'][i+1]   
  cond3 = df['high'][i+1] > df['high'][i+2]   
  cond4 = df['high'][i-1] > df['high'][i-2]  
  return (cond1 and cond2 and cond3 and cond4)

# to make sure the new level area does not exist already
def _is_far_from_level(value, unique_levels, df):
    # Clean noise in data by discarding a level if it is near another
    # (i.e. if distance to the next level is less than the average candle size for any given day - this will give a rough estimate on volatility) 
    ave =  np.mean(df['high'] - df['low'])    
    return np.sum([abs(value-level)<ave for _,level in unique_levels])==0

# This function, given a price value, returns True or False depending on if it is too near to some previously discovered key level.
def _distance_from_mean(mean, level, levels):
    return np.sum([abs(level - y) < mean for y in levels]) == 0

def remove_noise(df, levels, ltp):
    # Clean noise in data by discarding a level if it is near another
    # (i.e. if distance to the next level is less than the average candle size for any given day - this will give a rough estimate on volatility)
    mean = np.mean(df['high'] - df['low'])
    
    unique_levels = []
    for l in levels:
        if _distance_from_mean(mean, l, unique_levels):
            unique_levels.append(l)
    return unique_levels
      
#method 1: fractal candlestick pattern
def _fractal_candlestick_pattern_sr(df):
  levels = []  
  indexes = list(df.index.values)
  for i in range(2,df.shape[0]-2):
    index = Timestamp(indexes[i])
    if _is_support(df,i):
      l = df['low'][i]
      if _is_far_from_level(l, levels, df):
        levels.append((index,l))
    elif _is_resistance(df,i):
      l = df['high'][i]
      if _is_far_from_level(l, levels, df):
        levels.append((index,l))
  return levels

def fractal_candlestick_pattern_sr_2(df, n1=2, n2=2):
  levels = []  
  indexes = list(df.index.values)
  for i in range(2,df.shape[0]-2):
    index = Timestamp(indexes[i])
    if _support(df,i, n1, n2):
      l = df['low'][i]
      if _is_far_from_level(l, levels, df):
        levels.append((index,l))
    elif _resistance(df,i, n1, n2):
      l = df['high'][i]
      if _is_far_from_level(l, levels, df):
        levels.append((index,l))
  return levels

#method 2: window shifting method
def window_shifting_method_sr(df, window=5):
  levels = []
  max_list = []
  min_list = []
  for i in range(window, len(df)-window):
      high_range = df['high'][i-window:i+window-1]
      current_max = high_range.max()
      if current_max not in max_list:
          max_list = []
      max_list.append(current_max)
      if len(max_list) == window and _is_far_from_level(current_max, levels, df):
          levels.append((high_range.idxmax(), current_max))
      
      low_range = df['low'][i-window:i+window]
      current_min = low_range.min()
      if current_min not in min_list:
          min_list = []
      min_list.append(current_min)
      if len(min_list) == window and _is_far_from_level(current_min, levels, df):
          levels.append((low_range.idxmin(), current_min))
  return levels

def get_support_resistance(df, n1=2, n2=2, window=5):
    #n1 n2 before and after candle index 
    all_pivots_dict = []
    levels = fractal_candlestick_pattern_sr_2(df, n1, n2)
    for level in levels:
        point = np.round(level[1], 2)
        
        pivot = {}
        pivot["type"] = "SR - fractal candlestick pattern"
        pivot["date"]=level[0].to_pydatetime()
        pivot["level"] = point
        all_pivots_dict.append(pivot)

    levels = window_shifting_method_sr(df, window)
    for level in levels:
        point = np.round(level[1], 2)
        
        pivot = {}
        pivot["type"] = "SR - window shifting method"
        pivot["date"]=level[0].to_pydatetime()
        pivot["level"] = point
        all_pivots_dict.append(pivot)
        
    return all_pivots_dict

def _find_nearest_index(levels, value):
    array = np.asarray(levels)
    idx = (np.abs(levels - value)).argmin()
    return idx

def shrink_list(levels, ltp, items_count=10):
    idx = _find_nearest_index(levels, ltp)
    
    min_idx = idx-items_count
    max_idx = idx+items_count
    
    if min_idx < 0:
        min_idx = 0
        
    if max_idx > len(levels)-1:
        max_idx = len(levels)-1
        
    return levels[min_idx:max_idx]


Data Collection

In [43]:
def get_stock_price(symbol, period="2y", interval="1d", start_date=None, end_date=None):
  df = yf.download(tickers=symbol, interval=interval, period=period, start=start_date, end=end_date)
  df['Date'] = pd.to_datetime(df.index)
  df['Date'] = df['Date'].apply(mpl_dates.date2num)
  df = df.loc[:,['Date', 'Open', 'high', 'low', 'Close']]
  return df

In [100]:
# symbol = 'TCS.NS'
# symbol = '^NSEBANK'
symbol = "^NSEI"
df_y = get_stock_price(symbol, "2y", "1d")
df_h = get_stock_price(symbol, "6mo", "1h")
df_15m = get_stock_price(symbol, "1mo", "15m")
df_5m = get_stock_price(symbol, "7d", "5m")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [101]:
df_y = sanitize(df_y)
df_h = sanitize(df_h)
df_15m = sanitize(df_15m)
df_5m = sanitize(df_5m)

In [106]:
levels = []

ml = monthly_levels(df_y)
levels.extend(ml)
#print([x["level"] for x in ml])

ml = weekly_levels(df_y)
levels.extend(ml)
#print([x["level"] for x in ml])

ml = firty_two_week_levels(df_y)
levels.extend(ml)
#print([x["level"] for x in ml])

ml = get_support_resistance(df_y)
levels.extend(ml)
#print([x["level"] for x in ml])

ml = get_support_resistance(df_h)
levels.extend(ml)
#print([x["level"] for x in ml])

ml = get_support_resistance(df_15m)
levels.extend(ml)
#print([x["level"] for x in ml])

l = [x["level"] for x in levels]

price = df_5m.iloc[-1]["close"]

l = remove_noise(df_15m, l, price)
l.sort()

print(price)
print(shrink_list(l, price))

18311.25
[17959.0, 17989.15, 18023.0, 18049.95, 18087.95, 18130.7, 18175.4, 18211.75, 18254.35, 18282.0, 18311.0, 18362.0, 18399.0]
